[View in Colaboratory](https://colab.research.google.com/github/praveenkanumarlapudi/SQL-Playground/blob/master/RANK_DENSE_RANK_FIRST_LAST_SQL.ipynb)

# RANK, DENSE_RANK, FIRST and LAST Analytic Functions

In [0]:
--DROP TABLE emp PURGE;

CREATE TABLE emp (
  empno    NUMBER(4) CONSTRAINT pk_emp PRIMARY KEY,
  ename    VARCHAR2(10),
  job      VARCHAR2(9),
  mgr      NUMBER(4),
  hiredate DATE,
  sal      NUMBER(7,2),
  comm     NUMBER(7,2),
  deptno   NUMBER(2)
);

INSERT INTO emp VALUES (7369,'SMITH','CLERK',7902,to_date('17-12-1980','dd-mm-yyyy'),800,NULL,20);
INSERT INTO emp VALUES (7499,'ALLEN','SALESMAN',7698,to_date('20-2-1981','dd-mm-yyyy'),1600,300,30);
INSERT INTO emp VALUES (7521,'WARD','SALESMAN',7698,to_date('22-2-1981','dd-mm-yyyy'),1250,500,30);
INSERT INTO emp VALUES (7566,'JONES','MANAGER',7839,to_date('2-4-1981','dd-mm-yyyy'),2975,NULL,20);
INSERT INTO emp VALUES (7654,'MARTIN','SALESMAN',7698,to_date('28-9-1981','dd-mm-yyyy'),1250,1400,30);
INSERT INTO emp VALUES (7698,'BLAKE','MANAGER',7839,to_date('1-5-1981','dd-mm-yyyy'),2850,NULL,30);
INSERT INTO emp VALUES (7782,'CLARK','MANAGER',7839,to_date('9-6-1981','dd-mm-yyyy'),2450,NULL,10);
INSERT INTO emp VALUES (7788,'SCOTT','ANALYST',7566,to_date('13-JUL-87','dd-mm-rr')-85,3000,NULL,20);
INSERT INTO emp VALUES (7839,'KING','PRESIDENT',NULL,to_date('17-11-1981','dd-mm-yyyy'),5000,NULL,10);
INSERT INTO emp VALUES (7844,'TURNER','SALESMAN',7698,to_date('8-9-1981','dd-mm-yyyy'),1500,0,30);
INSERT INTO emp VALUES (7876,'ADAMS','CLERK',7788,to_date('13-JUL-87', 'dd-mm-rr')-51,1100,NULL,20);
INSERT INTO emp VALUES (7900,'JAMES','CLERK',7698,to_date('3-12-1981','dd-mm-yyyy'),950,NULL,30);
INSERT INTO emp VALUES (7902,'FORD','ANALYST',7566,to_date('3-12-1981','dd-mm-yyyy'),3000,NULL,20);
INSERT INTO emp VALUES (7934,'MILLER','CLERK',7782,to_date('23-1-1982','dd-mm-yyyy'),1300,NULL,10);
COMMIT;

**RANK :** RANK() is used when you want to rank the records based on a column values.

Syntax:

RANK() OVER ([query_partition_clause] [orderby_clause[window_clause]])

Example: Find the rank of the employees based on their salaries PER EACH DEPT

In [0]:
SELECT EMPNO, DEPTNO, SAL, 
                      RANK() OVER (PARTITION BY DEPTNO ORDER BY SAL) EMP_RANK
              FROM EMP

In [0]:
RESULT:
  
EMPNO	DEPTNO	SAL	EMP_RANK
7934	10	1300	1
7782	10	2450	2
7839	10	5000	3
7369	20	800	  1
7876	20	1100	2
7566	20	2975	3
7788	20	3000	4
7902	20	3000	4
7900	30	950	  1
7654	30	1250	2
7521	30	1250	2
7844	30	1500	4
7499	30	1600	5
7698	30	2850	6

**Observations :**
* When two people are sharing same rank the next consicutive rank is skipped as shown in                                  above example 7564 and 7521 has the same rank and the next consicutive rank 3 is  skipped, rank 4 is assigned to next record.

In [0]:
Example : Find Top 2 lowest paid Employees dept wise
  SELECT * FROM
(SELECT EMPNO, DEPTNO, SAL,
         RANK() OVER (PARTITION BY DEPTNO ORDER BY SAL) EMP_RANK
         FROM EMP)
WHERE EMP_RANK <= 2

In [0]:
EMPNO	DEPTNO	SAL	EMP_RANK
7934	10	1300	1
7782	10	2450	2
7369	20	800	  1
7876	20	1100	2
7900	30	950	  1
7521	30	1250	2
7654	30	1250	2

**DENSE_RANK:**
  Dense rank is similar to rank except that it assigns consicutive rankings even after duplicates
  
  Example : Find the rank of the employees according to their pay dept wise

In [0]:
select empno, deptno, sal,
   DENSE_RANK() OVER (PARTITION BY deptno ORDER BY sal) emp_rank
   from emp

In [0]:
EMPNO	DEPTNO	SAL	EMP_RANK
7934	10	1300	1
7782	10	2450	2
7839	10	5000	3
7369	20	800	1
7876	20	1100	2
7566	20	2975	3
7788	20	3000	4
7902	20	3000	4
7900	30	950	1
7654	30	1250	2
7521	30	1250	2
7844	30	1500	3
7499	30	1600	4
7698	30	2850	5

**Observations : **
* You can see even after two ranks are repeted next consicutive rank is given

**FIRST and LAST** :
First and Last can be used to return First and Last value with in a Partition Group.

Example: 
Lets say you want to display Lowest and Highest Salaries along with other details dept wise

In [0]:
SELECT DEPTNO, EMPNO, SAL,
   MIN(SAL) KEEP (DENSE_RANK FIRST ORDER BY SAL) OVER (PARTITION BY DEPTNO) AS LOWEST,
   MAX(SAL) KEEP (DENSE_RANK LAST ORDER BY SAL) OVER (PARTITION BY DEPTNO) AS HIGHEST
   FROM EMP

In [0]:
DEPTNO	EMPNO	SAL	LOWEST	HIGHEST
10	7782	2450	1300	5000
10	7839	5000	1300	5000
10	7934	1300	1300	5000
20	7566	2975	800	3000
20	7902	3000	800	3000
20	7876	1100	800	3000
20	7369	800	800	3000
20	7788	3000	800	3000
30	7521	1250	950	2850
30	7844	1500	950	2850
30	7499	1600	950	2850
30	7900	950	950	2850
30	7698	2850	950	2850
30	7654	1250	950	2850

SMILAR RESULT CAN BE ACHIVED USING FIRST_VALUE AND LAST_VALUE

In [0]:
SELECT EMPNO, DEPTNO, SAL, FIRST_VALUE(SAL IGNORE NULLS)
                           OVER (PARTITION BY DEPTNO ORDER BY SAL RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) LOWEST,
                           LAST_VALUE(SAL IGNORE NULLS) 
                           OVER (PARTITION BY DEPTNO ORDER BY SAL RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) HIGHEST
                           FROM EMP

In [0]:
EMPNO	DEPTNO	SAL	LOWEST	HIGHEST
7934	10	1300	1300	5000
7782	10	2450	1300	5000
7839	10	5000	1300	5000
7369	20	800	800	3000
7876	20	1100	800	3000
7566	20	2975	800	3000
7788	20	3000	800	3000
7902	20	3000	800	3000
7900	30	950	950	2850
7654	30	1250	950	2850
7521	30	1250	950	2850
7844	30	1500	950	2850
7499	30	1600	950	2850
7698	30	2850	950	2850